In [ ]:

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")



# Add the Pytorch-HarDNet repository to the Python path
sys.path.append('./Pytorch-HarDNet_E2')
from hardnet import HarDNet



# Dataset paths
DATA_DIR = "/app//"
TRAIN_DIR = os.path.join(DATA_DIR, "train")
VAL_DIR = os.path.join(DATA_DIR, "val")
TEST_DIR = os.path.join(DATA_DIR, "test")

# Hyperparameters
BATCH_SIZE = 16
NUM_CLASSES = 2
NUM_EPOCHS = 50
LEARNING_RATE = 0.0001

# Data augmentation and normalization
transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomApply([transforms.ColorJitter(brightness=0.2, contrast=0.2)], p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load datasets
data = {
    'train': datasets.ImageFolder(TRAIN_DIR, transform=transform['train']),
    'val': datasets.ImageFolder(VAL_DIR, transform=transform['val']),
    'test': datasets.ImageFolder(TEST_DIR, transform=transform['test'])
}

dataloaders = {
    'train': DataLoader(data['train'], batch_size=BATCH_SIZE, shuffle=True),
    'val': DataLoader(data['val'], batch_size=BATCH_SIZE, shuffle=False),
    'test': DataLoader(data['test'], batch_size=BATCH_SIZE, shuffle=False)
}

# Define HarDNet85 model from scratch (no pretrained weights, no fine-tuning)
def get_hardnet85_from_scratch():
    model = HarDNet( arch=85, pretrained=False)
    model.base[-1][3] = nn.Linear(model.base[-1][3].in_features, NUM_CLASSES)
    model.to(device)
    return model

model = get_hardnet85_from_scratch()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Early stopping utility
class EarlyStopping:
    def __init__(self, patience=5):
        self.patience = patience
        self.best_loss = float('inf')
        self.counter = 0

    def step(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Training loop with early stopping and history tracking
def train_model_with_early_stopping(model, dataloaders, criterion, optimizer, num_epochs=NUM_EPOCHS):
    best_model_wts = model.state_dict()
    best_acc = 0.0
    early_stopper = EarlyStopping(patience=5)

    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': []
    }

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == 'train':
                history['train_loss'].append(epoch_loss)
                history['train_acc'].append(epoch_acc.item())
            else:
                history['val_loss'].append(epoch_loss)
                history['val_acc'].append(epoch_acc.item())

                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = model.state_dict()

                if early_stopper.step(epoch_loss):
                    print("⚠️ Early stopping triggered.")
                    model.load_state_dict(best_model_wts)
                    return model, history

    model.load_state_dict(best_model_wts)
    return model, history

# Train model
model, history = train_model_with_early_stopping(model, dataloaders, criterion, optimizer)

# Save best model
os.makedirs("/app/", exist_ok=True)
torch.save(model.state_dict(), "/app//hardnetE2.pth")
print("✅ Model trained and saved as 'hardnetE2.pth'")

# Plot training curves
def plot_training_curves(history, save_path):
    # Loss plot
    plt.figure(figsize=(8, 6))
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_path, "loss_curve.png"))
    plt.show()

    # Accuracy plot
    plt.figure(figsize=(8, 6))
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Val Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_path, "accuracy_curve.png"))
    plt.show()

plot_training_curves(history, save_path="/app/")

# Evaluation
def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_labels), np.array(all_preds)

# Confusion matrix plotting
def plot_confusion_matrix(labels, preds, phase):
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=data['train'].classes, yticklabels=data['train'].classes)
    plt.title(f'{phase} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

# Run evaluation
for phase in ['train', 'val', 'test']:
    labels, preds = evaluate_model(model, dataloaders[phase])
    print(f"\n📊 {phase.capitalize()} Classification Report:\n", classification_report(labels, preds, target_names=data['train'].classes, digits=4))
    plot_confusion_matrix(labels, preds, phase)
